In [1]:
from src.manager import application_manager

In [2]:
manager = application_manager.ApplicationManager()

In [3]:
manager.apiManager.retrieve_mod("the twilight forrest")

name: the twilight forrest
url:  https://api.modrinth.com/v2/search?query=the+twilight+forrest&limit=20&index=relevance&offset=0&facets=%5B%5B%22project_type+%3A+mod%22%5D%5D
- indices :  [10] 
- scores :  [85.3]
best projects:  ['The Aether II']
redirecting to api:  search_id_tag
url:  https://api.modrinth.com/v2/projects?ids=%5B%22JD2NSu5O%22%5D
redirecting to api:  id_tag
url:  https://www.curseforge.com/api/v1/mods/search?filterText=the+twilight+forrest&gameId=432&sortField=1&pageSize=50&index=0
- indices :  [0] 
- scores :  [96.9]
best projects:  ['The Twilight Forest']
redirecting to api:  search_id_tag
url:  https://api.curseforge.com/v1/mods
redirecting to api:  id_tag


TypeError: unsupported operand type(s) for -: 'list' and 'list'